In [615]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [616]:
def loader(file_path, separator='\t', header=0):
    """
    Charge un fichier de données (CSV ou TXT)
    
    Parameters:
    -----------
    file_path : str
        Chemin vers le fichier
    separator : str
        Séparateur utilisé dans le fichier ('\t' pour tab, ',' pour virgule, ' ' pour espace, etc.)
    header : int or None
        Ligne à utiliser comme en-têtes (0 = première ligne, None = pas d'en-tête)
    """
    try:
        # Essayer de détecter automatiquement le format
        return pd.read_csv(file_path, sep=separator, header=header)
    except:
        # Si ça échoue, essayer avec d'autres séparateurs communs
        separators = ['\t', ',', ';', ' ', '|']
        for sep in separators:
            try:
                return pd.read_csv(file_path, sep=sep, header=header)
            except:
                continue
        raise ValueError(f"Impossible de lire le fichier {file_path}. Format non reconnu.")

In [617]:
def loader_tracking(file_path, separator='\t', header=None):
    """
    Charge un fichier de données (CSV ou TXT) et nomme les colonnes frame, player, x, y
    
    Parameters:
    -----------
    file_path : str
        Chemin vers le fichier
    separator : str
        Séparateur utilisé dans le fichier ('\t' pour tab, ',' pour virgule, ' ' pour espace, etc.)
    header : int or None
        Ligne à utiliser comme en-têtes (None = pas d'en-tête par défaut)
    """
    # Noms des colonnes prédéfinis
    column_names = ['frame', 'player', 'x', 'y']
    
    try:
        # Essayer de détecter automatiquement le format
        df = pd.read_csv(file_path, sep=separator, header=header, names=column_names)
        return df
    except:
        # Si ça échoue, essayer avec d'autres séparateurs communs
        separators = ['\t', ',', ';', ' ', '|']
        for sep in separators:
            try:
                df = pd.read_csv(file_path, sep=sep, header=header, names=column_names)
                return df
            except:
                continue
        raise ValueError(f"Impossible de lire le fichier {file_path}. Format non reconnu.")

In [618]:
def lps_sample(df):
    df_sample = df.copy()
    df_sample = df_sample[::2]
    return df_sample

In [619]:
def tracking_sample(df):
    """
    Échantillonne le DataFrame en gardant une frame sur 5
    """
    df_sample = df.copy()
    
    # Obtenir les frames uniques et les trier
    unique_frames = sorted(df_sample['frame'].unique())
    
    # Prendre une frame sur 5 (indices 0, 5, 10, 15, ...)
    selected_frames = unique_frames[::5]
    
    # Filtrer le DataFrame pour ne garder que les frames sélectionnées
    df_sample = df_sample[df_sample['frame'].isin(selected_frames)]
    
    return df_sample


In [620]:
INPUT_LPS_DIRECTORY = "C:/Users/Rémi/Documents/stage/stage_Dynateam/Stage_DynaTeam/basket/LPS"
OUTPUT_LPS_DIRECTORY = "C:/Users/Rémi/Documents/stage/stage_Dynateam/Stage_DynaTeam/basket/LPS_sample"

In [621]:
def process_directory_lps(input_directory, output_directory):
    """
    Traite tous les fichiers CSV d'un répertoire avec lps_sample et les sauvegarde.
    
    Parameters:
    -----------
    input_directory : str
        Chemin vers le répertoire contenant les fichiers CSV à traiter
    output_directory : str
        Chemin vers le répertoire de sortie (par défaut: le chemin spécifié)
    
    Returns:
    --------
    dict : Dictionnaire avec les résultats du traitement
    """
    
    # Convertir les chemins en objets Path pour une meilleure gestion
    input_path = Path(input_directory)
    output_path = Path(output_directory)
    
    # Créer le répertoire de sortie s'il n'existe pas
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Dictionnaire pour stocker les résultats
    results = {
        'processed_files': [],
        'errors': [],
        'total_files': 0,
        'successful': 0
    }
    
    # Chercher tous les fichiers CSV dans le répertoire d'entrée
    csv_files = list(input_path.glob("*.csv"))
    results['total_files'] = len(csv_files)
    
    if not csv_files:
        print(f"Aucun fichier CSV trouvé dans {input_directory}")
        return results
    
    print(f"Traitement de {len(csv_files)} fichiers CSV...")
    
    for csv_file in csv_files:
        try:
            print(f"Traitement de {csv_file.name}...")
            
            # Charger le fichier avec votre fonction loader
            df = loader(str(csv_file))
            
            # Appliquer l'échantillonnage LPS
            df_sampled = lps_sample(df)
            
            # Créer le nom du fichier de sortie
            output_filename = f"SAMPLE_{csv_file.name}"
            output_filepath = output_path / output_filename
            
            # Reset de l'index (correction ici)
            df_sampled = df_sampled.reset_index(drop=True)
            
            # Sauvegarder le fichier échantillonné
            df_sampled.to_csv(output_filepath, index=False)
            print(df_sampled)
            
            # Enregistrer le succès
            results['processed_files'].append({
                'input_file': str(csv_file),
                'output_file': str(output_filepath),
                'original_rows': len(df),
                'sampled_rows': len(df_sampled)
            })
            results['successful'] += 1
            
            print(f"  ✓ Sauvegardé: {output_filename} ({len(df)} → {len(df_sampled)} lignes)")
            
        except Exception as e:
            error_info = {
                'file': str(csv_file),
                'error': str(e)
            }
            results['errors'].append(error_info)
            print(f"  ✗ Erreur avec {csv_file.name}: {e}")
    
    # Afficher le résumé
    print(f"\n=== RÉSUMÉ ===")
    print(f"Fichiers traités avec succès: {results['successful']}/{results['total_files']}")
    print(f"Erreurs: {len(results['errors'])}")
    print(f"Fichiers sauvegardés dans: {output_directory}")
    
    return results

In [622]:
results = process_directory_lps(INPUT_LPS_DIRECTORY, OUTPUT_LPS_DIRECTORY)


Traitement de 11 fichiers CSV...
Traitement de 10_fusion.csv...
      timestamp;lat_brute;long_brute;latitude_fusion;longitude_fusion;hdop;vitesse_fusion;battery
0                          1702478113100;;;;;-1.0;-1.0;0                                         
1      1702478113300;;;49.4580534274501;1.06233783223...                                         
2      1702478113500;;;49.4580513443479;1.06234140907...                                         
3      1702478113700;;;49.4580491138085;1.06234443824...                                         
4      1702478113900;;;49.4580467999047;1.06234699240...                                         
...                                                  ...                                         
43734  1702486859900;;;49.4580478495537;1.06241392862...                                         
43735  1702486860100;;;49.4580484118619;1.06241183518...                                         
43736  1702486860300;;;49.4580486013132;1.06240998994.

In [623]:
INPUT_TRACKING_DIRECTORY = "C:/Users/Rémi/Documents/stage/stage_Dynateam/Stage_DynaTeam/basket/TRACKING"
OUTPUT_TRACKING_DIRECTORY = "C:/Users/Rémi/Documents/stage/stage_Dynateam/Stage_DynaTeam/basket/TRACKING_sample"

In [624]:
def process_directory_tracking(input_directory, 
                         output_directory,
                         file_extension="txt",
                         separator='\t',
                         header=0):
    """
    Traite tous les fichiers de données d'un répertoire avec lps_sample et les sauvegarde.
    
    Parameters:
    -----------
    input_directory : str
        Chemin vers le répertoire contenant les fichiers à traiter
    output_directory : str
        Chemin vers le répertoire de sortie
    file_extension : str
        Extension des fichiers à traiter ("txt", "csv", "dat", etc.)
    separator : str
        Séparateur utilisé dans les fichiers ('\t', ',', ';', ' ', etc.)
    header : int or None
        Ligne à utiliser comme en-têtes (0 = première ligne, None = pas d'en-tête)
    
    Returns:
    --------
    dict : Dictionnaire avec les résultats du traitement
    """
    
    # Convertir les chemins en objets Path pour une meilleure gestion
    input_path = Path(input_directory)
    output_path = Path(output_directory)
    
    # Créer le répertoire de sortie s'il n'existe pas
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Dictionnaire pour stocker les résultats
    results = {
        'processed_files': [],
        'errors': [],
        'total_files': 0,
        'successful': 0
    }
    
    # Chercher tous les fichiers avec l'extension spécifiée
    pattern = f"*.{file_extension}"
    data_files = list(input_path.glob(pattern))
    results['total_files'] = len(data_files)
    
    if not data_files:
        print(f"Aucun fichier .{file_extension} trouvé dans {input_directory}")
        return results
    
    print(f"Traitement de {len(data_files)} fichiers .{file_extension}...")
    
    for data_file in data_files:
        try:
            print(f"Traitement de {data_file.name}...")
            
            # Charger le fichier avec votre fonction loader modifiée
            df = loader_tracking(str(data_file), separator=separator, header=header)
            
            # Vérifier que le DataFrame n'est pas vide
            if df.empty:
                print(f"  ⚠ Fichier vide: {data_file.name}")
                continue
            
            # Appliquer l'échantillonnage LPS
            df_sampled = tracking_sample(df)
            
            # Créer le nom du fichier de sortie (en gardant l'extension .txt ou .csv selon vos préférences)
            output_filename = f"SAMPLE_{data_file.stem}.csv"  # .stem enlève l'extension
            output_filepath = output_path / output_filename
            
            # Sauvegarder le fichier échantillonné
            df_sampled.to_csv(output_filepath, index=True, sep=separator)
            
            # Enregistrer le succès
            results['processed_files'].append({
                'input_file': str(data_file),
                'output_file': str(output_filepath),
                'original_rows': len(df),
                'sampled_rows': len(df_sampled)
            })
            results['successful'] += 1
            
            print(f"  ✓ Sauvegardé: {output_filename} ({len(df)} → {len(df_sampled)} lignes)")
            
        except Exception as e:
            error_info = {
                'file': str(data_file),
                'error': str(e)
            }
            results['errors'].append(error_info)
            print(f"  ✗ Erreur avec {data_file.name}: {e}")
    
    # Afficher le résumé
    print(f"\n=== RÉSUMÉ ===")
    print(f"Fichiers traités avec succès: {results['successful']}/{results['total_files']}")
    print(f"Erreurs: {len(results['errors'])}")
    print(f"Fichiers sauvegardés dans: {output_directory}")
    
    return results


In [625]:
results = process_directory_tracking(INPUT_TRACKING_DIRECTORY, OUTPUT_TRACKING_DIRECTORY)

Traitement de 190 fichiers .txt...
Traitement de Equipe1_Vague1_Poss10_video_1.txt...
  ✓ Sauvegardé: SAMPLE_Equipe1_Vague1_Poss10_video_1.csv (3027 → 611 lignes)
Traitement de Equipe1_Vague1_Poss10_video_3.txt...
  ✓ Sauvegardé: SAMPLE_Equipe1_Vague1_Poss10_video_3.csv (2901 → 583 lignes)
Traitement de Equipe1_Vague1_Poss11_video_1.txt...
  ✓ Sauvegardé: SAMPLE_Equipe1_Vague1_Poss11_video_1.csv (5023 → 1006 lignes)
Traitement de Equipe1_Vague1_Poss11_video_3.txt...
  ✓ Sauvegardé: SAMPLE_Equipe1_Vague1_Poss11_video_3.csv (4967 → 994 lignes)
Traitement de Equipe1_Vague1_Poss12_video_1.txt...
  ✓ Sauvegardé: SAMPLE_Equipe1_Vague1_Poss12_video_1.csv (4326 → 867 lignes)
Traitement de Equipe1_Vague1_Poss12_video_3.txt...
  ✓ Sauvegardé: SAMPLE_Equipe1_Vague1_Poss12_video_3.csv (4232 → 843 lignes)
Traitement de Equipe1_Vague1_Poss13_video_1.txt...
  ✓ Sauvegardé: SAMPLE_Equipe1_Vague1_Poss13_video_1.csv (4234 → 843 lignes)
Traitement de Equipe1_Vague1_Poss13_video_3.txt...
  ✓ Sauvegardé: S

In [626]:
ar = np.array([[1.1, 2, 3.3, 4], [2.7, 10, 5.4, 7], [5.3, 9, 1.5, 15]])
df = pd.DataFrame(ar, index = ['a1', 'a2', 'a3'])

In [627]:
lps_sample(df)

,0,1,2,3
a1,1.1,2.0,3.3,4.0
a3,5.3,9.0,1.5,15.0


In [628]:
df = loader("C:/Users/Rémi/Documents/stage/stage_Dynateam/Stage_DynaTeam/basket/TRACKING/Equipe1_Vague1_Poss2_video_1.txt", ",")
tracking_sample

<function __main__.tracking_sample(df)>